In [1]:
!pip install duckdb

In [2]:
import duckdb

In [3]:
con = duckdb.connect(database=':memory:')

In [4]:
con.execute("CREATE TABLE input AS "
            "SELECT id as rid, concat(given_name, ' ', surname) as val "
            "FROM 'data/S1_clean_.csv'")

In [5]:
con.execute("insert into input "
            "select id, concat(given_name, ' ', surname) "
            "from 'data/S2_clean_.csv'")

In [6]:
con.execute("select * from input")
con.fetchall()

[('S1_0', 'joshua morrison'),
 ('S1_1', 'jordan white'),
 ('S1_2', 'emmerson lock'),
 ('S1_3', 'alexandra grosser'),
 ('S1_4', 'michael wuchatsch'),
 ('S1_5', 'emmerson loyck'),
 ('S1_6', 'rhys schuetz'),
 ('S1_7', 'joshua greenj'),
 ('S1_8', 'olivia hobson'),
 ('S1_9', 'michael lierach'),
 ('S1_10', 'elisabett domiten'),
 ('S1_11', 'genoveffa hylander'),
 ('S2_0', 'braecon schuetz'),
 ('S2_1', 'alexandra grosvenor'),
 ('S2_2', 'michael liersch'),
 ('S2_3', 'emmeron loyk'),
 ('S2_4', 'olivia hobson'),
 ('S2_5', 'joshua green'),
 ('S2_6', 'charlotte hyland'),
 ('S2_7', 'elisabet domitienn')]

In [7]:
q = 3

In [8]:
maxlen = con.execute("select max(length(val)) from input").fetchall()[0][0]

In [9]:
con.execute(
    "create table tokens as "
    f"select rid, length(val) + {q} - 1 as rlen, "
    f"x as pos, substring(concat(repeat('#', {q} - 1), "
    "lower(val), "
    f"repeat('#',{q} - 1)),"
    f"x, {q}) as token "
    f"from input, range(1, {maxlen + q}) tbl(x) "
    f"where x <= length(val) + {q} - 1"
)

In [10]:
con.execute("select * from tokens").fetchall()

[('S1_1', 14, 1, '##j'),
 ('S1_1', 14, 2, '#jo'),
 ('S1_1', 14, 3, 'jor'),
 ('S1_1', 14, 4, 'ord'),
 ('S1_1', 14, 5, 'rda'),
 ('S1_1', 14, 6, 'dan'),
 ('S1_1', 14, 7, 'an '),
 ('S1_1', 14, 8, 'n w'),
 ('S1_1', 14, 9, ' wh'),
 ('S1_1', 14, 10, 'whi'),
 ('S1_1', 14, 11, 'hit'),
 ('S1_1', 14, 12, 'ite'),
 ('S1_1', 14, 13, 'te#'),
 ('S1_1', 14, 14, 'e##'),
 ('S1_6', 14, 1, '##r'),
 ('S1_6', 14, 2, '#rh'),
 ('S1_6', 14, 3, 'rhy'),
 ('S1_6', 14, 4, 'hys'),
 ('S1_6', 14, 5, 'ys '),
 ('S1_6', 14, 6, 's s'),
 ('S1_6', 14, 7, ' sc'),
 ('S1_6', 14, 8, 'sch'),
 ('S1_6', 14, 9, 'chu'),
 ('S1_6', 14, 10, 'hue'),
 ('S1_6', 14, 11, 'uet'),
 ('S1_6', 14, 12, 'etz'),
 ('S1_6', 14, 13, 'tz#'),
 ('S1_6', 14, 14, 'z##'),
 ('S2_3', 14, 1, '##e'),
 ('S2_3', 14, 2, '#em'),
 ('S2_3', 14, 3, 'emm'),
 ('S2_3', 14, 4, 'mme'),
 ('S2_3', 14, 5, 'mer'),
 ('S2_3', 14, 6, 'ero'),
 ('S2_3', 14, 7, 'ron'),
 ('S2_3', 14, 8, 'on '),
 ('S2_3', 14, 9, 'n l'),
 ('S2_3', 14, 10, ' lo'),
 ('S2_3', 14, 11, 'loy'),
 ('S2_3', 14,

In [11]:
con.execute("CREATE TABLE doc_frequency AS "
            "SELECT token, count(*) AS df "
            "FROM tokens "
            "GROUP BY token "
            # "ORDER BY 2"
            # Save one ORDER BY clause here, it will be performed later
            )
con.execute("SELECT * FROM doc_frequency").fetchall()

[('##j', 4),
 ('#jo', 4),
 ('jor', 1),
 ('ord', 1),
 ('rda', 1),
 ('dan', 1),
 ('an ', 1),
 ('n w', 1),
 (' wh', 1),
 ('whi', 1),
 ('hit', 1),
 ('ite', 2),
 ('te#', 1),
 ('e##', 1),
 ('##r', 1),
 ('#rh', 1),
 ('rhy', 1),
 ('hys', 1),
 ('ys ', 1),
 ('s s', 1),
 (' sc', 2),
 ('sch', 4),
 ('chu', 2),
 ('hue', 2),
 ('uet', 2),
 ('etz', 2),
 ('tz#', 2),
 ('z##', 2),
 ('##e', 5),
 ('#em', 3),
 ('emm', 3),
 ('mme', 3),
 ('mer', 3),
 ('ero', 1),
 ('ron', 1),
 ('on ', 4),
 ('n l', 3),
 (' lo', 3),
 ('loy', 2),
 ('oyk', 1),
 ('yk#', 1),
 ('k##', 3),
 ('jos', 3),
 ('osh', 3),
 ('shu', 3),
 ('hua', 3),
 ('ua ', 3),
 ('a g', 4),
 (' gr', 4),
 ('gre', 2),
 ('ree', 2),
 ('een', 2),
 ('en#', 2),
 ('n##', 6),
 ('ers', 3),
 ('rso', 2),
 ('son', 5),
 ('loc', 1),
 ('ock', 1),
 ('ck#', 2),
 ('enj', 1),
 ('nj#', 1),
 ('j##', 1),
 ('##o', 2),
 ('#ol', 2),
 ('oli', 2),
 ('liv', 2),
 ('ivi', 2),
 ('via', 2),
 ('ia ', 2),
 ('a h', 3),
 (' ho', 2),
 ('hob', 2),
 ('obs', 2),
 ('bso', 2),
 ('on#', 3),
 ('oyc', 1),

In [12]:
# Keep this select query to compare the output against the Table R, which will not keep 'df'
con.execute(
    "SELECT tt.*, df.df "
    "FROM tokens AS tt "
    "JOIN doc_frequency AS df ON tt.token = df.token "
    "ORDER BY tt.rid, df.df"
).fetchall()

[('S1_0', 17, 11, 'orr', 1),
 ('S1_0', 17, 8, 'a m', 1),
 ('S1_0', 17, 9, ' mo', 1),
 ('S1_0', 17, 10, 'mor', 1),
 ('S1_0', 17, 14, 'iso', 1),
 ('S1_0', 17, 13, 'ris', 1),
 ('S1_0', 17, 12, 'rri', 1),
 ('S1_0', 17, 4, 'osh', 3),
 ('S1_0', 17, 3, 'jos', 3),
 ('S1_0', 17, 5, 'shu', 3),
 ('S1_0', 17, 6, 'hua', 3),
 ('S1_0', 17, 7, 'ua ', 3),
 ('S1_0', 17, 16, 'on#', 3),
 ('S1_0', 17, 2, '#jo', 4),
 ('S1_0', 17, 1, '##j', 4),
 ('S1_0', 17, 15, 'son', 5),
 ('S1_0', 17, 17, 'n##', 6),
 ('S1_1', 14, 5, 'rda', 1),
 ('S1_1', 14, 6, 'dan', 1),
 ('S1_1', 14, 14, 'e##', 1),
 ('S1_1', 14, 13, 'te#', 1),
 ('S1_1', 14, 11, 'hit', 1),
 ('S1_1', 14, 10, 'whi', 1),
 ('S1_1', 14, 9, ' wh', 1),
 ('S1_1', 14, 8, 'n w', 1),
 ('S1_1', 14, 3, 'jor', 1),
 ('S1_1', 14, 4, 'ord', 1),
 ('S1_1', 14, 7, 'an ', 1),
 ('S1_1', 14, 12, 'ite', 2),
 ('S1_1', 14, 2, '#jo', 4),
 ('S1_1', 14, 1, '##j', 4),
 ('S1_10', 19, 9, 'ett', 1),
 ('S1_10', 19, 17, 'ten', 1),
 ('S1_10', 19, 10, 'tt ', 1),
 ('S1_10', 19, 14, 'omi', 2),


In [13]:
con.execute(
    "CREATE TABLE R AS ("
    "SELECT tt.* "
    "FROM tokens AS tt "
    "JOIN doc_frequency AS df ON tt.token = df.token "
    "ORDER BY tt.rid, df.df"
    ")"
)

In [14]:
con.execute("select * from R").fetchall()

[('S1_0', 17, 11, 'orr'),
 ('S1_0', 17, 8, 'a m'),
 ('S1_0', 17, 9, ' mo'),
 ('S1_0', 17, 10, 'mor'),
 ('S1_0', 17, 14, 'iso'),
 ('S1_0', 17, 13, 'ris'),
 ('S1_0', 17, 12, 'rri'),
 ('S1_0', 17, 4, 'osh'),
 ('S1_0', 17, 3, 'jos'),
 ('S1_0', 17, 5, 'shu'),
 ('S1_0', 17, 6, 'hua'),
 ('S1_0', 17, 7, 'ua '),
 ('S1_0', 17, 16, 'on#'),
 ('S1_0', 17, 2, '#jo'),
 ('S1_0', 17, 1, '##j'),
 ('S1_0', 17, 15, 'son'),
 ('S1_0', 17, 17, 'n##'),
 ('S1_1', 14, 5, 'rda'),
 ('S1_1', 14, 6, 'dan'),
 ('S1_1', 14, 14, 'e##'),
 ('S1_1', 14, 13, 'te#'),
 ('S1_1', 14, 11, 'hit'),
 ('S1_1', 14, 10, 'whi'),
 ('S1_1', 14, 9, ' wh'),
 ('S1_1', 14, 8, 'n w'),
 ('S1_1', 14, 3, 'jor'),
 ('S1_1', 14, 4, 'ord'),
 ('S1_1', 14, 7, 'an '),
 ('S1_1', 14, 12, 'ite'),
 ('S1_1', 14, 2, '#jo'),
 ('S1_1', 14, 1, '##j'),
 ('S1_10', 19, 9, 'ett'),
 ('S1_10', 19, 17, 'ten'),
 ('S1_10', 19, 10, 'tt '),
 ('S1_10', 19, 14, 'omi'),
 ('S1_10', 19, 3, 'eli'),
 ('S1_10', 19, 5, 'isa'),
 ('S1_10', 19, 18, 'en#'),
 ('S1_10', 19, 15, 'mit'),

In [15]:
t = 0.5

In [16]:
# Keep this select query to compare the output against the Table PrefixR, which will not keep 'rown'
con.execute(
    "select rid, rlen, pos, token, nrow "
    "from ("
    "SELECT R.*, row_number() OVER (PARTITION BY rid) as nrow "
    "FROM R "
    ")"
    f"where nrow <= rlen - ceil(rlen * {t}) + 1"
).fetchall()

[('S1_0', 17, 11, 'orr', 1),
 ('S1_0', 17, 8, 'a m', 2),
 ('S1_0', 17, 9, ' mo', 3),
 ('S1_0', 17, 10, 'mor', 4),
 ('S1_0', 17, 14, 'iso', 5),
 ('S1_0', 17, 13, 'ris', 6),
 ('S1_0', 17, 12, 'rri', 7),
 ('S1_0', 17, 4, 'osh', 8),
 ('S1_0', 17, 3, 'jos', 9),
 ('S1_1', 14, 5, 'rda', 1),
 ('S1_1', 14, 6, 'dan', 2),
 ('S1_1', 14, 14, 'e##', 3),
 ('S1_1', 14, 13, 'te#', 4),
 ('S1_1', 14, 11, 'hit', 5),
 ('S1_1', 14, 10, 'whi', 6),
 ('S1_1', 14, 9, ' wh', 7),
 ('S1_1', 14, 8, 'n w', 8),
 ('S1_10', 19, 9, 'ett', 1),
 ('S1_10', 19, 17, 'ten', 2),
 ('S1_10', 19, 10, 'tt ', 3),
 ('S1_10', 19, 14, 'omi', 4),
 ('S1_10', 19, 3, 'eli', 5),
 ('S1_10', 19, 5, 'isa', 6),
 ('S1_10', 19, 18, 'en#', 7),
 ('S1_10', 19, 15, 'mit', 8),
 ('S1_10', 19, 13, 'dom', 9),
 ('S1_10', 19, 4, 'lis', 10),
 ('S1_11', 20, 9, 'ffa', 1),
 ('S1_11', 20, 8, 'eff', 2),
 ('S1_11', 20, 7, 'vef', 3),
 ('S1_11', 20, 6, 'ove', 4),
 ('S1_11', 20, 5, 'nov', 5),
 ('S1_11', 20, 3, 'gen', 6),
 ('S1_11', 20, 2, '#ge', 7),
 ('S1_11', 20, 

In [34]:
# no primary key on (id, token) yet
con.execute(
    "create table PrefixR as "
    "select rid, rlen, pos, token "
    "from ("
    "SELECT R.*, row_number() OVER (PARTITION BY rid) as nrow "
    "FROM R "
    ")"
    f"where nrow <= rlen - ceil(rlen * {t}) + 1"
)

CatalogException: Catalog Error: Table with name "PrefixR" already exists!

In [35]:
x=con.execute("select * from PrefixR").fetchall()
len(x)

182

In [36]:
#this min_len is the alpha value in slide of "Set similarity join", the records with a len less than this value don't be used

min_len = 15

In [44]:
con.execute(
    "CREATE TABLE candidate AS("
    "SELECT DISTINCT PR1.rid AS rid1, PR2.rid AS rid2 "
    "FROM PrefixR PR1, PrefixR PR2 "
    "WHERE PR1.rid < PR2.rid "
    "AND PR1.pos = PR2.pos "
    f"AND PR1.rlen - PR1.pos >= CEIL(PR1.rlen * 2 * {t} / (1+{t})) "
    f"AND (PR1.rlen >= {min_len} AND PR2.rlen >= {min_len}) "
    f"AND LEAST((PR1.rlen - PR1.pos), (PR2.rlen - PR2.pos)) >= CEIL( (PR1.rlen + PR2.rlen) * {t} / ( 1 + {t} ))) "
    )

In [47]:
con.execute("SELECT * FROM CANDIDATE").fetchall()

[('S1_0', 'S2_6'),
 ('S1_0', 'S2_7'),
 ('S1_10', 'S2_7'),
 ('S1_10', 'S2_6'),
 ('S1_11', 'S2_7'),
 ('S1_11', 'S2_4'),
 ('S1_2', 'S2_7'),
 ('S1_3', 'S2_4'),
 ('S1_3', 'S2_7'),
 ('S1_3', 'S2_6'),
 ('S1_4', 'S2_7'),
 ('S1_4', 'S2_6'),
 ('S1_5', 'S2_7'),
 ('S1_7', 'S2_7'),
 ('S1_7', 'S2_6'),
 ('S1_8', 'S2_6'),
 ('S1_8', 'S2_7'),
 ('S1_8', 'S2_4'),
 ('S1_9', 'S2_7'),
 ('S2_0', 'S2_6'),
 ('S2_0', 'S2_7'),
 ('S2_0', 'S2_4'),
 ('S2_1', 'S2_6'),
 ('S2_1', 'S2_7'),
 ('S2_2', 'S2_4'),
 ('S2_2', 'S2_7'),
 ('S2_4', 'S2_7'),
 ('S2_4', 'S2_6'),
 ('S2_6', 'S2_7'),
 ('S1_0', 'S2_4'),
 ('S1_10', 'S2_4'),
 ('S1_11', 'S2_6'),
 ('S1_11', 'S2_2'),
 ('S1_2', 'S2_6'),
 ('S1_2', 'S2_4'),
 ('S1_3', 'S2_2'),
 ('S1_4', 'S2_4'),
 ('S1_5', 'S2_6'),
 ('S1_7', 'S2_4'),
 ('S1_8', 'S2_2'),
 ('S1_9', 'S2_6'),
 ('S2_0', 'S2_2'),
 ('S2_1', 'S2_4'),
 ('S2_2', 'S2_6'),
 ('S1_0', 'S2_1'),
 ('S1_10', 'S2_1'),
 ('S1_11', 'S2_1'),
 ('S1_11', 'S2_0'),
 ('S1_2', 'S2_1'),
 ('S1_3', 'S2_0'),
 ('S1_5', 'S2_4'),
 ('S1_7', 'S2_1'),
 (